In [1]:
import ctypes
import time
import os
import multiprocessing

_CCnt = ctypes.CDLL('./CCnt.so')

## Testing signed vs unsigned 32-bit integers

In [21]:
signed = _CCnt.getCC()
unsigned = signed
if unsigned < 0:
    unsigned += 2**32
print(signed)
print(unsigned)

-1610404447
2684562849


In [4]:
t1 = 4294967295
t2 = 1
t3 = t2-t1
t3 += 2**32
print(t3)

2


In [2]:
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))


In [3]:
def timed_fib(CPU,n):
    os.system("taskset -pc {} {}".format(CPU,p_fib.pid))
    
    _CCnt.init(1,0) #initialize/reset counter
    
    t_before = time.time()
    cc_before = _CCnt.getCC()

    recur_fibo(n)
        
    cc_after = _CCnt.getCC()
    t_after = time.time()

    #convert to unsigned
    if cc_after < 0:
        cc_after += 2**32
    if cc_before < 0:
        cc_before += 2**32
        
    elapsed_time = t_after-t_before
    num_cycles = cc_after-cc_before
    
    #account for possibility of overflow
    if num_cycles < 0:
        num_cycles += 2**32
    
    print('Cycle count before: {}'.format(cc_before))
    print('Cycle count after: {}'.format(cc_after))
    print('Number of Cycles: {}'.format(num_cycles))
    print('Elapsed Time: {} sec'.format(elapsed_time))
    
    print('Calculated time based on CPU Frequency: {} sec'.format(num_cycles/650000000))

## Run recur_fibo on a specific CPU:

In [4]:
n = 30
CPU = 1

#NOTE: This must be named p_fib in order to set the CPU affinity correctly
p_fib = multiprocessing.Process(target=timed_fib, args=(CPU,n))
p_fib.start()
p_fib.join()

Cycle count before: 73983
Cycle count after: 2420086484
Number of Cycles: 2420012501
Elapsed Time: 3.723316192626953 sec
Calculated time based on CPU Frequency: 3.7230961553846154 sec
